# This notebook is to underestand easily for especially train data.
## (ver6 adding submission test and explanation)
## if it is helpful for you, please upvote!

In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

# 0. Confirming each csv file

In [ ]:
traindf = pd.read_csv("../input/shopee-product-matching/train.csv")
traindf

In [ ]:
testdf = pd.read_csv("../input/shopee-product-matching/test.csv")
testdf

In [ ]:
sample = pd.read_csv("../input/shopee-product-matching/sample_submission.csv")
sample

In [ ]:
TRAIN_PATH = "../input/shopee-product-matching/train_images"
TEST_PATH = "../input/shopee-product-matching/test_images"

traindf["path"] = [os.path.join(TRAIN_PATH,s) for s in traindf["image"]]
testdf["path"] = [os.path.join(TEST_PATH,s) for s in testdf["image"]]

traindf.to_csv("traindf.csv",index=False)
testdf.to_csv("testdf.csv",index=False)

# 1. Train.csv

In [ ]:
# path列が追加されたのを確認できる
traindf.head(3)

In [ ]:
traindf.info()

* no nan data

In [ ]:
for col in traindf.columns:
    print(col + ":" + str(len(traindf[col].unique())))

* This result shows the how many uniques in each columns.
* Total rows are 34250. Each column has some duplicates.
* There are duplicates in images, hashes, and titles, not just label groups.

## 1.1 easy visualizing image

In [ ]:
img = cv2.imread(traindf["path"].iloc[0]) # iloc[0]で1枚目の画像を指定している
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# traindf["label_group"].iloc[0] -> traindfの"label_group"列の1番目の値を取得している
# つまり、traindfの"label_group"列の1番目の値「249114794」と同じ値をもつ行を取得している。
tmpdf = traindf[traindf["label_group"]==traindf["label_group"].iloc[0]]
tmpdf

In [ ]:
# 「label_group」が同じだが、表示される画像は異なる（場合もある）
for a in tmpdf["path"]:
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(img)
    plt.axis("off")

## 1.2 Understanding same label_groups

### 1.2.1 Counting the number of images in each label group

In [ ]:
labels = traindf.groupby("label_group")["image"].count().reset_index()
labels.columns=["label_group","image_num"]
labels

In [ ]:
# label_groupが同じ物が最大で51個存在する。また、最小の物は2個である。
# -> label_groupは予測時に手助けになりそう。
sortlabels = labels.sort_values("image_num")
sortlabels

### 1.2.2 Counting the label_groups in each image_num

In [ ]:
# 出力結果1行目：label_groupが同じ物が2つのみのグループのトータル数
# 出力結果2行目：label_groupが同じ物が3つのみのグループのトータル数
imagecount = labels.groupby("image_num").count().reset_index()
imagecount.columns=["image_num","counts"]
imagecount

In [ ]:
# ほとんどの画像はgroup_labelの同値が2つのみの場合が多い
plt.bar(imagecount["image_num"],imagecount["counts"])

### 1.2.3 visualizing images of label_group with max counts 

In [ ]:
traindf.head(5)

In [ ]:
sortlabels.tail(5)

In [ ]:
# sortlabels["label_group"].iloc[-1]は上記に表示されている通り、label_groupが同値のものが51個存在するグループである
# 中にはimageおよphashが同じ物が確認できる
tmpdf = traindf[traindf["label_group"]==sortlabels["label_group"].iloc[-1]]
tmpdf.head(51)
    

In [ ]:
len(tmpdf)

In [ ]:
# label_groupが同値のものが51個存在する1グループを表示する。

plt.figure(figsize=(20,20))

for num,a in enumerate(tmpdf["path"]):
    plt.subplot(11,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(img)
    
    

## 1.3 Understanding same images

In [ ]:
imagegroup = traindf.groupby("image")["path"].count().reset_index()
imagegroup.columns=["image","counts"]
imagegroup

In [ ]:
# imageの値が同値の物が多数存在する（phashが同値とは限らない）
tmpdf = imagegroup[imagegroup["counts"] > 1]
tmpdf.sort_values("counts")
tmpdf

In [ ]:
# 試しに3つ以上物があるかを確認してみる -> 存在する（最大カウントは15）
tmpdf_3 = imagegroup[imagegroup["counts"] > 10]
tmpdf_3.sort_values("counts")
tmpdf_3

In [ ]:
# imageの値が最大な「0cca4afba97e106abd0843ce72881ca4.jpg」の15枚の画像を表示してみる
# imageの値が同じならphashの値も等しい。ただし、titleが異なったり、label_groupが異なることもある。。。
tmp_ex = traindf[traindf["image"]=="0cca4afba97e106abd0843ce72881ca4.jpg"]
tmp_ex

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(tmp_ex["path"]):
    plt.subplot(5,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(img)

上記により、同じ画像だが（titleもほとんど同じだが）、2つのラベルグループに分類されることがわかった
それぞれのラベルグループに属する画像を表示してみる

In [ ]:
# label_group「2403374241」に属するものを抽出すると、複数のphash値が確認できた（複数種類の画像が存在する）
tmp_a = traindf[traindf["label_group"]==2403374241]
tmp_a

In [ ]:
# 実際に画像を表示してみる
plt.figure(figsize=(20,20))

for num,a in enumerate(tmp_a["path"]):
    plt.subplot(5,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(img)

In [ ]:
# phash値が同じなので全て同じ画像ということがわかる
tmp_b = traindf[traindf["label_group"]==4198148727]
tmp_b

In [ ]:
# 実際に画像を表示してみる
plt.figure(figsize=(20,20))

for num,a in enumerate(tmp_b["path"]):
    plt.subplot(5,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(img)

同じ画像なのに（phashが同値）、複数のlabel_groupがあるものを確認する

In [ ]:
imagegroup = traindf.groupby(["image_phash","label_group"])["path"].count().reset_index()
imagegroup.columns=["image_phash","label_group","count"]
imagegroup

In [ ]:
# duplicated(keep=False)は抽出した画像を抜き出すもの。かつ、両方残す方法
tmp = imagegroup[imagegroup["image_phash"].duplicated(keep=False)]
tmp.sort_values("image_phash")
tmp

In [ ]:
tmp["count"].sum()

In [ ]:
imagegroup = traindf.groupby(["image_phash","title","label_group",])["path"].count().reset_index()
imagegroup.columns=["image_phash","title","label_group","count"]
imagegroup

In [ ]:
# duplicated(keep=False)は抽出した画像を抜き出すもの。かつ、両方残す方法
tmp = imagegroup[imagegroup["image_phash"].duplicated(keep=False)]
tmp.sort_values("image_phash")
tmp

In [ ]:
tmp["count"].sum()

imageの同値数が2番目に多い物についても検証

In [ ]:
# こちらについては、imageが同値だとphashも同値で、titleは違うがlabel_groupも同じであった
tmp_ex = traindf[traindf["image"]=="c739a327dbeca472089a5195e898cce4.jpg"]
tmp_ex

In [ ]:
# 画像の表示
plt.figure(figsize=(20,20))

for num,a in enumerate(tmp_ex["path"]):
    plt.subplot(5,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.imshow(img)

In [ ]:
tmpdf["image"].iloc[0]

In [ ]:
# posting_id以外は同値である
traindf[traindf["image"]==tmpdf["image"].iloc[0]]

In [ ]:
# しかしこちらは、posting_idだけではなく、titleまで異なっている（imageとphashが同じにも関わらず）
traindf[traindf["image"]==tmpdf["image"].iloc[1]]

In [ ]:
# imageとphashが同じもの　-> 32412件存在する。
traindf.groupby(["image","image_phash"])["path"].count().reset_index()

## 1.4 Understanding same image_phash

In [ ]:
# phashが同値のグループをカウントしている（pathでカウントしているが、他でカウントしても問題ない）
phashgroup = traindf.groupby("image_phash")["path"].count().reset_index()
phashgroup.columns=["image_phash","counts"]
phashgroup

In [ ]:
# phashが同値の物が最大で26個存在することが確認できる（imageが同値の物が最大で15だった。。。）
# つまり、phashの精度はかなり低い？？それとも、imageの値が異なるが、実際は同じ画像（phashは同じになると想定）の物が多数存在するのか
sortphash = phashgroup.sort_values("counts")
sortphash

In [ ]:
# sortphash["image_phash"].iloc[-1]はphashが「fad28daa2ad05595」で、同値の物が26個存在する
# やはり、phashが同値だが、imageの値が異なるものが多数存在する
# また、titleも異なるが、label_groupは全て同値
# -> label_groupの括りの信用度は高め？
tmpdf = traindf[traindf["image_phash"]==sortphash["image_phash"].iloc[-1]]
print(len(tmpdf))
tmpdf

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(tmpdf["path"]): # tmpdfの行数は26個（phash値が全て同じもの）
    plt.subplot(6,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.axis("off")
    plt.title(tmpdf["posting_id"].iloc[num])
    plt.imshow(img)
    

上記を見るに、phashが同じものは、やはり画像自体も同じように思える（細かくは違うのか？）

仮に微小な違いがあったとしても、imageの値が違かったり、またtitleの値が異なる（後者は人間が付与してるから？そうするとtitleの信用度は低い？）

## 1.5 Understanding same titles

In [ ]:
traindf

In [ ]:
# ユニークなtitleは33117個存在する。
titlegroup = traindf.groupby("title")["path"].count().reset_index()
titlegroup.columns=["title","counts"]
titlegroup

In [ ]:
# titleが同値の物の最大値は9個
sorttitle = titlegroup.sort_values("counts")
sorttitle

In [ ]:
# sorttitle["title"].iloc[-1] -> titleの同値が9個もつ値
tmpdf = traindf[traindf["title"]==sorttitle["title"].iloc[-1]]
tmpdf

上記をみるに、titleが同値でも、image、phashまたgroup_labelすら異なることが多いと思われる

つまり、title個別の信用度はかなり低いのでは？

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(tmpdf["path"]):
    plt.subplot(3,5,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.axis("off")
    plt.imshow(img)
    

上記をみるに、titleが同値だが、画像が明らかに違うのが確認できる。ただし、雰囲気は同じ感じ！

# 2. test.csv

In [ ]:
testdf

In [ ]:
plt.figure(figsize=(20,20))

for num,a in enumerate(testdf["path"]):
    
    plt.subplot(1,3,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.axis("off")
    

# 3.submission.csv

In [ ]:
sample

## submission rules

sample_submission.csv - a sample submission file in the correct format.

posting_id - the ID code for the posting.

matches - Space delimited list of all posting IDs that match this posting. Posts always self-match. Group sizes were capped at 50, so there's no need to predict more than 50 matches.

# 4.Easy test to submission
### From above results, I thought it would match if the image_phash were the same.⇨同意見
### To achieve that, I used defaultdict, but firstly I practiced with train.csv because test.csv is too short and difficult to understand.

In [ ]:
# 1000行まで抽出。列に関しては全て
practicedf = traindf.iloc[:1000,:]

In [ ]:
practicedf

In [ ]:
from collections import defaultdict

In [ ]:
every_phash = defaultdict(list)

In [ ]:
for num, row in enumerate(practicedf[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
every_phash

In [ ]:
every_phash_list = []

for num, row in enumerate(practicedf[['posting_id','image_phash']].values):
    pred = ""
    for a in every_phash[row[1]]:
        pred = pred + a + " "

    
    pred=pred[:-1] # delete last space
    
    every_phash_list.append(pred)

In [ ]:
every_phash_list[:20]

In [ ]:
practicedf["matches"] = every_phash_list
practicedf

## The case using test data to submit

In [ ]:
testdf

In [ ]:
every_phash = defaultdict(list)

In [ ]:
for num, row in enumerate(testdf[['posting_id', 'image_phash']].values):
    every_phash[row[1]].append(row[0])

In [ ]:
every_phash

In [ ]:
every_phash_list = []

for num, row in enumerate(testdf[['posting_id','image_phash']].values):
    pred = ""
    for a in every_phash[row[1]]:
        pred = pred + a + " "

    
    pred=pred[:-1] # delete last space
    
    every_phash_list.append(pred)

In [ ]:
testdf["matches"] = every_phash_list
testdf

In [ ]:
submission = testdf[["posting_id","matches"]]
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

## Thank you for reading!! Attension : internet must be off.